In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='0'


import spacy
nlp = spacy.load('fr_dep_news_trf')

In [9]:
import sys
sys.path.append('..')
from extract_corpus import get_coref_corpus

corpus = get_coref_corpus()


In [17]:
doc = nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")
for tok in doc:
  print(f"{tok.text:<10} {tok.dep_:<10} {tok.pos_}")

Mme        nsubj      NOUN
JOUDKIA    flat:name  PROPN
est        cop        AUX
jolie      ROOT       ADJ
.          punct      PUNCT
Elle       nsubj      PRON
mange      ROOT       VERB
des        det        DET
fleurs     obj        NOUN
.          punct      PUNCT
Monsieur   nsubj      NOUN
LEBORGNE   flat:name  PROPN
est        cop        AUX
malade     ROOT       ADJ
.          punct      PUNCT
Il         nsubj      PRON
est        ROOT       VERB
à          case       ADP
l'         det        DET
hopital    obl:arg    NOUN
.          punct      PUNCT


In [20]:

# for year in corpus:
#   print(year)

for year in ["1814"]:
# for year in corpus:
  year_text = "\n".join([chunk.text for chunk in corpus[year]]) #le but c'est de concatener le texte de chaque chunk de 1917
  doc = nlp(year_text)
  adjectifs = [token for token in doc if token.pos_ == "ADJ"]
  for adj in adjectifs:
    for child in adj.children:
        if child.dep_ == "nsubj":
          subj = None
          for chunk in doc.noun_chunks:
            if (chunk[0].i <= child.i) and (child.i <= chunk[-1].i):
              subj = chunk
          if not subj:
            subj = child
          print(f"{adj.text:<20} || {subj.text:<20} || {subj.}")
          print("=================")


même                 || MM. Désaugiers et Gentil, dont M. Gosse auteur du Médisant, comédie reçue antérieurement à l’Hôtel Garni prétend que le sujet est le même que le M. Gosse auteur du Médisant, comédie reçue antérieurement à l’Hôtel Garni || []
disponibles          || les 100 F            || []
disponible           || la gratification     || []
disponibles          || les 100 F            || []
inamovibles          || Les deux doyens      || []
rééligibles          || Les quatre autres membres || []
nécessaires          || qui                  || []
agréable             || le témoignage        || []
relatif              || M. Cartigny          || []
ancien               || M. Ferriere          || []
ancien               || M. Ferriere arrêté à M. Baudron, chef, qui atteste que M. Ferriere est le plus ancien des surnuméraires || []
égal                 || M. Cartigny          || []
relatif              || qui                  || []
incompatibles        || les fonctions        || []


AttributeError: 'spacy.tokens.token.Token' object has no attribute 'ents'

In [ ]:
year_text = "\n".join([chunk.text for chunk in corpus['1814']]) #le but c'est de concatener le texte de chaque chunk de 1917
doc = nlp(year_text)

In [26]:
doc =  nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")

adjectifs = [token for token in doc if token.pos_ == "ADJ"]
for adj in adjectifs:
  for child in adj.children:
      if child.dep_ == "nsubj":
        subj = None
        for chunk in doc.noun_chunks:
          if (chunk[0].i <= child.i) and (child.i <= chunk[-1].i):
            subj = chunk
        if not subj:
          subj = child
        print(f"{adj.text:<20} || {subj.text:<20} || {subj.ents}")
        print("=================")



persons = [ent for ent in doc.ents if ent.label_ == "PER"]
for person in persons:
  if (person.root.head.pos_ == "ADJ" and person.root.dep_=="nsubj"):
    print(f"{person.text:<20} {person.root.head.text:<20} {person.root.head.pos_:<10} {person.root.dep_}")
print("=================")

jolie                || Mme JOUDKIA          || []
malade               || Monsieur LEBORGNE    || []


In [92]:

nlp = spacy.load("fr_core_news_lg")
doc = nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade et gros. Il est à l'hopital.")

adjectives_subjects = {}
def compute_adjectives_subjects(doc):
	local_adj_subj = {}
	for ent in doc.ents:
		if ent.label_ == "PER":
			local_adj_subj[ent] = []

	for adjective in local_adj_subj.keys():
		for adj_tok in adjective:
				if (adj_tok.head.pos_ == "ADJ"):
					local_adj_subj[adjective].append(adj_tok.head)
    for subject, adjs in local_adj_subj:
        print("ok")
					

compute_adjectives_subjects(doc)
compute_adjectives_subjects(doc)
for subject, adjs in adjectives_subjects.items():
		print(f"{subject} -> {adjs}")


In [83]:
for tok in doc:
    print(f"{tok.text:<20} {tok.dep_:<20} {tok.pos_:<20}")


Mme                  nsubj:pass           NOUN                
JOUDKIA              flat:name            PROPN               
est                  cop                  AUX                 
jolie                ROOT                 ADJ                 
.                    punct                PUNCT               
Elle                 nsubj                PRON                
mange                ROOT                 VERB                
des                  case                 ADP                 
fleurs               nmod                 NOUN                
.                    punct                PUNCT               
Monsieur             nsubj                NOUN                
LEBORGNE             nmod                 PROPN               
est                  cop                  AUX                 
malade               ROOT                 ADJ                 
et                   cc                   CCONJ               
gros                 conj                 ADJ          

# test with stanza

In [19]:
import stanza

nlp = stanza.Pipeline("fr")

doc = nlp("Mme JOUDKIA est jolie. Elle mange des fleurs. Monsieur LEBORGNE est malade. Il est à l'hopital.")

ModuleNotFoundError: No module named 'stanza'

In [ ]:

def recursive_find_adjs(root, sent):
    children = [w for w in sent.words if w.head == root.id]

    if not children:
        return []

    filtered_c = [w for w in children if w.deprel == "conj" and w.upos == "ADJ"]
    # Do not include an adjective if it is the parent of a noun to prevent
    results = [w for w in filtered_c if not any(sub.head == w.id and sub.upos == "NOUN" for sub in sent.words)]
    for w in children:
        results += recursive_find_adjs(w, sent)

    return results

for sent in doc.sentences:
    nouns = [w for w in sent.words if w.upos == "NOUN"]
    # print(nouns[0].pretty_print())
    # print("==")
    noun_adj_pairs = {}
    for noun in nouns:
        # Find constructions in the form of "La voiture est belle"
        # In this scenario, the adjective is the parent of the noun
        cop_root = sent.words[noun.head-1]
        adjs = [cop_root] + recursive_find_adjs(cop_root, sent) if cop_root.upos == "ADJ" else []

        # Find constructions in the form of "La femme intelligente et belle"
        # Here, the adjectives are descendants of the noun
        mod_adjs = [w for w in sent.words if w.head == noun.id and w.upos == "ADJ"]
        # This should only be one element because conjunctions are hierarchical
        if mod_adjs:
            mod_adj = mod_adjs[0]
            adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))

        if adjs:
            unique_adjs = []
            unique_ids = set()
            for adj in adjs:
                if adj.id not in unique_ids:
                    unique_adjs.append(adj)
                    unique_ids.add(adj.id)

            noun_adj_pairs[noun.text] = " ".join([adj.text for adj in unique_adjs])

    print(noun_adj_pairs)